In [2]:
import numpy as np
import scipy.special
from scipy.signal import convolve2d
import os
import os
import functools
import numpy as np
import scipy.special
from scipy.misc import imread
from skimage.color import rgb2gray
from scipy.signal import convolve2d
from scipy.ndimage import map_coordinates

In [3]:
np.dstack(np.meshgrid(np.arange(2), np.arange(5))).reshape(-1, 2)

array([[0, 0],
       [1, 0],
       [0, 1],
       [1, 1],
       [0, 2],
       [1, 2],
       [0, 3],
       [1, 3],
       [0, 4],
       [1, 4]])

In [7]:
a = np.arange(5)
b = np.arange(5,10)
a[:0]

array([], dtype=int64)

In [6]:
a[3:]

array([3, 4])